# Download the data using urllib2 and Beautifulsoup

In [92]:
import urllib2
from bs4 import BeautifulSoup
import os

In [ ]:
def create_relevant_links(page,relevant_text):
    page = urllib2.urlopen(page).read()
    relevant_links=list()
    for i in range(len(BeautifulSoup(page).find_all('a'))):
        if BeautifulSoup(page).find_all('a')[i]["href"].find(relevant_text)!=-1:
            relevant_links.append(BeautifulSoup(page).find_all('a')[i]["href"])
    return relevant_links

In [10]:
## get the links to download from
page='http://www.aueb.gr/users/ion/data/enron-spam/'
relevant_text="enron-spam/raw"
relevant_links=create_relevant_links(page,relevant_text)

In [20]:
## save files in system (raw/ham and raw/spam directories)

for link in relevant_links:
    f = urllib2.urlopen(link)
    data = f.read()
    if link.find('raw/ham/')!=-1:
        with open(os.path.join('raw/ham',link.partition('raw/ham/')[2]), "wb") as code:
            code.write(data)
            code.close()
    else:
        with open(os.path.join('raw/spam',link.partition('raw/spam/')[2]), "wb") as code:
            code.write(data)
            code.close()

# Process the data: untar, clean files, get pandas dataframe; write to csv

In [93]:
import re
import tarfile
import subprocess
import unidecode
import random
import pandas as pd
import nltk
from nltk.corpus import stopwords

In [195]:
def clean_html(file):
    # First remove inline JavaScript/CSS:
    your_text = file.partition('Subject:')[2]
    your_text = BeautifulSoup(your_text).get_text()
    ## Forwarded emails removed (necessary?)
    cleaned=your_text.partition('---------------------- Forwarded by')[0]
    ## Extra correction:
    cleaned = re.sub(r"(?is)<(script|style).*?>.*?(</\1>)", "", cleaned)
    a=[x for x in cleaned.split("\n") if (x!='' and  x!=' ' and x!='  '
                and x.find('Date:')==-1 and x.find('X-Mailer:')==-1 and x.find('Content-Type:')==-1
                and x.find('From:')==-1 and x.find('X-Priority:')==-1 and x.find('Content-Transfer-Encoding:')==-1
                and x.find('Content-Length:')==-1 and x.find('Lines:')==-1 and x.find('MIME')==-1
                and x.find('X-MimeOLE:')==-1 and x.find('X-MSMail-Priority:')==-1
                and x.find('Content-Description:')==-1 and x.find('X-Qmail-Scanner-Message-ID:')==-1
                and x.find('From:')==-1 and x.find('To:')==-1 and x.find('Message-ID:')==-1
                and x.find('Received:')==-1 and x.find('Return-Path:')==-1
                and x.find('Delivered-To:')==-1 and x.find('Processed in')==-1
                and x.find('Mime-Version:')==-1 and x.find('Cc:')==-1 and x.find('Bcc:')==-1
                and x.find('X-bcc:')==-1 and x.find('charset')==-1
                and x.find('boundary=:')==-1 and x.find('SMTP')==-1
                and x.find('Message-Id:')==-1 and x.find('@enron.com')==-1
                and x.find('X-Status:')==-1 and x.find('X-Keywords:')==-1
                and x.find('X-cc:')==-1 and x.find('X-Origin:')==-1 and x.find('X-FileName:')==-1)]
    result=  "\n".join(a)
    ##Encoding
    result = re.sub("[^a-zA-Z]", " ", result)
    result=unidecode.unidecode(result.decode('ascii'))
    result = re.sub("[^a-zA-Z]", " ", result)
    result = re.sub("  ", " ", result)
    result=result.lower()
    return result

In [26]:
## First untar the files
raw_path = ['raw/ham/','raw/spam/']

for path in raw_path:
    for root, dir, files in os.walk(path):
        print 'Files in ' + root[:]

    # untar tar.gz files one by one
        for file in files:
            print 'Processing ' + file
            file_path = root+file
            tar = tarfile.open(file_path)
            names = tar.getnames()
            tar.extractall(path)
            tar.close()            

Files in raw/ham/
Processing kaminski-v.tar.gz
Processing beck-s.tar.gz
Processing williams-w3.tar.gz
Processing kitchen-l.tar.gz
Processing lokay-m.tar.gz
Processing farmer-d.tar.gz
Done
Files in raw/spam/
Processing SH.tar.gz
Processing GP.tar.gz
Processing BG.tar.gz
Done


In [77]:
## Make a list of all text files present in untared files:
file_str = subprocess.check_output('find raw/ -type f', shell=True)
file_list = [ x for x in file_str.splitlines() if not x.endswith('tar.gz')]

## To make everything faster, let´s select 5000 elements randomly
random.seed(0)
sample = random.sample(file_list, 5000)

In [87]:
spam_number1=[1 if file_list[i].find('raw/ham/')==-1 else 0 for i in range(len(file_list))]
spam_number2=[1 if sample[i].find('raw/ham/')==-1 else 0 for i in range(len(sample))]
print "The whole list contains %d emails. %s percent of the whole list is spam." %(len(spam_number1),float(sum(spam_number1))/float(len(file_list))*100)
print "The sample list contains %d emails. %s percent of the sample list is spam." %(len(spam_number2),float(sum(spam_number2))/float(len(sample))*100)

## sample and the whole list are comparable
## to be sure, I also compared number of emails from different folders im sample and in the full list;
## they were comparable as well

The whole list contains 52076 emails. 63.3458790998 percent of the whole list is spam.
The sample list contains 5000 emails. 64.76 percent of the sample list is spam.


In [196]:
# recreate ham&spam directories in pre/
subprocess.check_call('sh remove_make_dir.sh', shell=True)

0

In [197]:
## safe cleaned sample in pre/ham and pre/spam

for i in sample:
    raw_html = open(i, 'r').read()
    cleaned_html = clean_html(raw_html)
    if i.find('raw/ham/')!=-1:
        with open(os.path.join('pre/ham',re.sub("/","_",i.partition('raw/ham/')[2])+'.txt'), "wb") as code:
            code.write(cleaned_html)
            code.close()
    else: 
        with open(os.path.join('pre/spam',re.sub("/","_",i.partition('raw/spam/')[2])+'.txt'), "wb") as code:
            code.write(cleaned_html)  
            code.close()

In [198]:
spam_path = 'pre/spam'
ham_path = 'pre/ham'
spam_dir = os.listdir(spam_path)
ham_dir = os.listdir(ham_path)
spam_filelist= [os.path.join(spam_path, f) for f in spam_dir]
ham_filelist = [os.path.join(ham_path, f) for f in ham_dir]

In [203]:
spam_text=[]
ham_text=[]
stops = set(stopwords.words("english")+['subject','message','mail','email','http','nextpart','rfkindysadvnqw','nerasdf'])

for i  in spam_filelist:
    spam_text.append(' '.join([e for e in open(i, 'r').read().split() if len(e)>2 and len(e)<20 and e not in stops]))
for i  in ham_filelist:
    ham_text.append(' '.join([e for e in open(i, 'r').read().split() if len(e)>2 and len(e)<20 and e not in stops]))
text=spam_text+ham_text
labels=[1]*len(spam_text)+[0]*len(ham_text)

In [204]:
df=pd.DataFrame(data={'text': text,'spam_ind': labels})
df.head(20)

,spam_ind,text
0,1,bargain prescripiton pills ephemeral command e...
1,1,chance get new logo user agent squirrelmail bo...
2,1,buy regalis also known superviagra cialis buy ...
3,1,cia llis softabs best boundary vxc page hello ...
4,1,shrugging shrugging cormier tra hroksd fbbgg b...
5,1,facilitate facilitate pardon abruptness libert...
6,1,medz foreveer boundary unsent hello england co...
7,1,best prices anywhere net seq uxlp never see do...
8,1,phamacy online storyteller boundary boundary g...
9,1,proscrastinate per year msmail priority normal...


In [205]:
df.to_csv('enron_data.csv',sep=';',index=False)